# Cart_Pole_Balancer_Project_for_AI_Foundation_ICA

# 1. Environment set up and Algorithm implementation

In [ ]:
! pip install "gym[classic_control]"

In [2]:
import os
import gym
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3 import PPO

In [3]:
env = gym.make('CartPole-v1')

In [4]:
gym.make('CartPole-v1')

<TimeLimit<CartPoleEnv<CartPole-v1>>>

# 2. Select a random Action

In [5]:
episodes = 10
for episode in range(episodes):
    state = env.reset()
    done = False
    reward_cnt = 0
    
    while not done:
        env.render()
        action = env.action_space.sample()
        new_state, reward, done, info = env.step(action)
        
        reward_cnt += reward
        state = new_state
    print(f"Episode: {episode} Score: {reward_cnt}")
env.close()

Episode: 0 Score: 21.0
Episode: 1 Score: 31.0
Episode: 2 Score: 42.0
Episode: 3 Score: 38.0
Episode: 4 Score: 26.0
Episode: 5 Score: 14.0
Episode: 6 Score: 33.0
Episode: 7 Score: 17.0
Episode: 8 Score: 45.0
Episode: 9 Score: 20.0


# 3 Algrithm Implementation and Training

In [6]:
log_path = os.path.join("train", "logs")

In [7]:
env = gym.make("CartPole-v1")
env = DummyVecEnv([lambda: env])
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path)

Using cpu device


In [8]:
model.learn(total_timesteps=10_000)

Logging to train\logs\PPO_66
-----------------------------
| time/              |      |
|    fps             | 245  |
|    iterations      | 1    |
|    time_elapsed    | 8    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 371         |
|    iterations           | 2           |
|    time_elapsed         | 11          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009123013 |
|    clip_fraction        | 0.112       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | 0.00194     |
|    learning_rate        | 0.0003      |
|    loss                 | 6.67        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0172     |
|    value_loss           | 49.1        |
-----------------------------------------
-----

# 4. Test running the agent 

In [9]:
episodes = 10
for episode in range(episodes):
    state = env.reset()
    done = False
    reward_cnt = 1
    
    while not done:
        env.render()
        action, _ = model.predict(state)
        new_state, reward, done, info = env.step(action)
        
        reward_cnt += reward
        state = new_state
    print(f"Episode: {episode} Score: {reward_cnt}")
env.close()

Episode: 0 Score: [156.]
Episode: 1 Score: [174.]
Episode: 2 Score: [154.]
Episode: 3 Score: [342.]
Episode: 4 Score: [171.]
Episode: 5 Score: [45.]
Episode: 6 Score: [148.]
Episode: 7 Score: [145.]
Episode: 8 Score: [133.]
Episode: 9 Score: [71.]


# 5. Hyper Parameter tuning

In [10]:
net_arch = [dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]

In [11]:
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path, policy_kwargs={"net_arch": net_arch})

Using cpu device


In [12]:
model.learn(total_timesteps=10_000)

Logging to train\logs\PPO_67
-----------------------------
| time/              |      |
|    fps             | 1093 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 699        |
|    iterations           | 2          |
|    time_elapsed         | 5          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.01583975 |
|    clip_fraction        | 0.248      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.681     |
|    explained_variance   | 0.0037     |
|    learning_rate        | 0.0003     |
|    loss                 | 3.29       |
|    n_updates            | 10         |
|    policy_gradient_loss | -0.0279    |
|    value_loss           | 18.2       |
----------------------------------------
-----------------------

# 6. Adding a Callbacks

In [13]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [14]:
save_path = os.path.join("train", "saved_models")

In [15]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=501, verbose=1)

In [16]:
eval_callback = EvalCallback(env,
                            callback_on_new_best=stop_callback,
                             best_model_save_path=save_path,
                             verbose=1)

# 7. Retraining

In [23]:
model = PPO("MlpPolicy", env=env, tensorboard_log=log_path, policy_kwargs={"net_arch": net_arch})

In [24]:
model.learn(total_timesteps=50_000, callback=eval_callback)

Eval num_timesteps=9760, episode_reward=408.80 +/- 111.95
Episode length: 408.80 +/- 111.95
Eval num_timesteps=19760, episode_reward=458.20 +/- 52.81
Episode length: 458.20 +/- 52.81
Eval num_timesteps=29760, episode_reward=500.00 +/- 0.00
Episode length: 500.00 +/- 0.00
New best mean reward!
Stopping training because the mean reward 500.00  is above the threshold 475


# 8. Save Model

In [25]:
MODEL_PATH = os.path.join("train", "saved_models", "PPO_cart_pole_model")

In [26]:
model.save(MODEL_PATH)

# 9. Reload Model

In [27]:
model = PPO.load(MODEL_PATH, env=env)

# 10. Evaluate Model

In [24]:
evaluate_policy(model, env=env, n_eval_episodes=10, render=True)


(467.7, 58.34046623056761)

# 11. Deployment

In [28]:
episodes = 10
for episode in range(episodes):
    state = env.reset()
    done = False
    reward_cnt = 1
    
    while not done:
        env.render()
        action, _ = model.predict(state)
        new_state, reward, done, info = env.step(action)
        
        reward_cnt += reward
        state = new_state
    print(f"Episode: {episode} Score: {reward_cnt}")
env.close()

Episode: 0 Score: [501.]
Episode: 1 Score: [501.]
Episode: 2 Score: [501.]
Episode: 3 Score: [375.]
Episode: 4 Score: [501.]
Episode: 5 Score: [501.]
Episode: 6 Score: [501.]
Episode: 7 Score: [501.]
Episode: 8 Score: [501.]
Episode: 9 Score: [501.]


# 12. View Performance

In [1]:
import tensorboard

In [6]:
training_log_path = os.path.join(log_path, "PPO_47")

In [9]:
training_log_path

'train\\logs\\PPO_47'

In [11]:
!tensorboard --logdir="./train/logs/"